In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
data = pd.read_csv("kaggle_visible_evaluation_triplets.txt", delimiter="\t", header=None, names=["user_id", "song_id", "play_count"])

print(data.head())
print(data.info())


                                    user_id             song_id  play_count
0  fd50c4007b68a3737fe052d5a4f78ce8aa117f3d  SOBONKR12A58A7A7E0           1
1  fd50c4007b68a3737fe052d5a4f78ce8aa117f3d  SOEGIYH12A6D4FC0E3           1
2  fd50c4007b68a3737fe052d5a4f78ce8aa117f3d  SOFLJQZ12A6D4FADA6           1
3  fd50c4007b68a3737fe052d5a4f78ce8aa117f3d  SOHTKMO12AB01843B0           1
4  fd50c4007b68a3737fe052d5a4f78ce8aa117f3d  SODQZCY12A6D4F9D11           1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450933 entries, 0 to 1450932
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   user_id     1450933 non-null  object
 1   song_id     1450933 non-null  object
 2   play_count  1450933 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 33.2+ MB
None


In [4]:
data

,user_id,song_id,play_count
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1
...,...,...,...
1450928,5e650759ebf89012044c6d52121eeada8b0ec814,SOVLNXV12A6D4F706E,1
1450929,5e650759ebf89012044c6d52121eeada8b0ec814,SOVDSJC12A58A7A271,2
1450930,5e650759ebf89012044c6d52121eeada8b0ec814,SOBRHVR12A8C133F35,2
1450931,5e650759ebf89012044c6d52121eeada8b0ec814,SOMGVYU12A8C1314FF,2


In [5]:
data['user_id'] = range(1, len(data) + 1)
data

,user_id,song_id,play_count
0,1,SOBONKR12A58A7A7E0,1
1,2,SOEGIYH12A6D4FC0E3,1
2,3,SOFLJQZ12A6D4FADA6,1
3,4,SOHTKMO12AB01843B0,1
4,5,SODQZCY12A6D4F9D11,1
...,...,...,...
1450928,1450929,SOVLNXV12A6D4F706E,1
1450929,1450930,SOVDSJC12A58A7A271,2
1450930,1450931,SOBRHVR12A8C133F35,2
1450931,1450932,SOMGVYU12A8C1314FF,2


In [6]:
# Sample a fraction of the data
data_sample = data.sample(frac=0.01, random_state=42)
user_item_matrix = data_sample.pivot_table(index="user_id", columns="song_id", values="play_count", fill_value=0).astype(np.uint8)

In [7]:
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)

In [8]:
user_similarity = cosine_similarity(train_data)

def recommend(user_id, n=5):
    # Check if the user_id is present in the index of the user_similarity matrix
    if user_id not in train_data.index:
        print(f"User {user_id} not found in the training data.")
        return []

    # Find the index corresponding to the user_id
    user_index = train_data.index.get_loc(user_id)

    user_row = user_similarity[user_index].reshape(1, -1)
    similar_users = user_similarity.argsort()[user_index][-n-1:-1]
    recommendations = []

    for user in similar_users:
        # Check if the user is present in the training data
        if user not in train_data.index:
            continue

        songs_played = train_data.loc[user].sort_values(ascending=False)
        unseen_songs = songs_played[songs_played == 0].index
        recommendations.extend(unseen_songs[:n])

    return recommendations


User 2 not found in the training data.
Recommended Songs for User 2:
[]


In [9]:
test_user_similarity = cosine_similarity(test_data)